# Using a new dataset

In this tutorial we show how you can use a dataset not present in the library.

This particular example uses the ENZIMES dataset, uses a simplicial lifting to create simplicial complexes, and trains the SCN2 model. We train the model using the appropriate training and validation datasets, and finally test it on the test dataset.

### <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
&emsp;[1. Imports](##sec1)

&emsp;[2. Configurations and utilities](##sec2)

&emsp;[3. Loading the data](##sec3)

&emsp;[4. Model initialization](##sec4)

&emsp;[5. Training](##sec5)

&emsp;[6. Testing the model](##sec6)

## 1. Imports <a class="anchor" id="sec1"></a>

In [10]:
import lightning as pl
import torch
from omegaconf import OmegaConf
from topomodelx.nn.simplicial.scn2 import SCN2
from torch_geometric.datasets import TUDataset

from topobenchmarkx.data.preprocess import PreProcessor
from topobenchmarkx.dataloader.dataloader import TBXDataloader
from topobenchmarkx.evaluator.evaluator import TBXEvaluator
from topobenchmarkx.loss.loss import TBXLoss
from topobenchmarkx.model.model import TBXModel
from topobenchmarkx.nn.encoders import AllCellFeatureEncoder
from topobenchmarkx.nn.readouts import PropagateSignalDown
from topobenchmarkx.nn.wrappers.simplicial import SCNWrapper

## 2. Configurations and utilities <a class="anchor" id="sec2"></a>

Configurations can be specified using yaml files or directly specified in your code like in this example.

In [11]:
transform_config = { "clique_lifting":
    {"transform_type": "lifting",
    "transform_name": "SimplicialCliqueLifting",
    "complex_dim": 3,}
}

split_config = {
    "learning_setting": "inductive",
    "split_type": "random",
    "data_seed": 0,
    "data_split_dir": "./data/ENZYMES/splits/",
    "train_prop": 0.5,
}

in_channels = 3
out_channels = 6
dim_hidden = 16

wrapper_config = {
    "out_channels": dim_hidden,
    "num_cell_dimensions": 3,
}

readout_config = {
    "readout_name": "PropagateSignalDown",
    "num_cell_dimensions": 1,
    "hidden_dim": dim_hidden,
    "out_channels": out_channels,
    "task_level": "graph",
    "pooling_type": "sum",
}

loss_config = {"task": "classification", "loss_type": "cross_entropy"}

evaluator_config = {"task": "classification",
                    "num_classes": out_channels,
                    "classification_metrics": ["accuracy", "precision", "recall"]}

transform_config = OmegaConf.create(transform_config)
split_config = OmegaConf.create(split_config)
readout_config = OmegaConf.create(readout_config)
loss_config = OmegaConf.create(loss_config)
evaluator_config = OmegaConf.create(evaluator_config)

In [12]:
def wrapper(**factory_kwargs):
    def factory(backbone):
        return SCNWrapper(backbone, **factory_kwargs)
    return factory

def scheduler(**factory_kwargs):
    def factory(optimizer):
        return torch.optim.lr_scheduler.StepLR(optimizer, **factory_kwargs)
    return factory

## 3. Loading the data <a class="anchor" id="sec3"></a>

In this example we use the ENZYMES dataset. It is a graph dataset and we use the clique lifting to transform the graphs into simplicial complexes. We invite you to check out the README of the [repository](https://github.com/pyt-team/TopoBenchmarkX) to learn more about the various liftings offered.

In [13]:
dataset_dir = "./data/ENZYMES/"
dataset = TUDataset(root=dataset_dir, name="ENZYMES")

preprocessor = PreProcessor(dataset, dataset_dir, transform_config)
dataset_train, dataset_val, dataset_test = preprocessor.load_dataset_splits(split_config)
datamodule = TBXDataloader(dataset_train, dataset_val, dataset_test, batch_size=32)

Processing...
Done!
Processing...
Done!


## 4. Model initialization <a class="anchor" id="sec4"></a>

We can create the backbone by instantiating the SCN2 model form TopoModelX. Then the `SCNWrapper` and the `TBXModel` take care of the rest.

In [14]:
backbone = SCN2(in_channels_0=dim_hidden, in_channels_1=dim_hidden, in_channels_2=dim_hidden)
wrapper = wrapper(**wrapper_config)

readout = PropagateSignalDown(**readout_config)
loss = TBXLoss(**loss_config)
feature_encoder = AllCellFeatureEncoder(in_channels=[in_channels, in_channels, in_channels], out_channels=dim_hidden)

evaluator = TBXEvaluator(**evaluator_config)
optimizer = torch.optim.Adam
scheduler = scheduler(step_size=50, gamma=0.5)

In [15]:
model = TBXModel(backbone=backbone,
                 backbone_wrapper=wrapper,
                 readout=readout,
                 loss=loss,
                 feature_encoder=feature_encoder,
                 evaluator=evaluator,
                 optimizer=optimizer,
                 scheduler=scheduler,
                 compile=False,)

## 5. Training <a class="anchor" id="sec5"></a>

Now we can use the `lightning` trainer to train the model.

In [19]:
%%capture
# Increase the number of epochs to get better results
trainer = pl.Trainer(max_epochs=5, accelerator="cpu", enable_progress_bar=False)

trainer.fit(model, datamodule)
train_metrics = trainer.callback_metrics

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name            | Type                  | Params
----------------------------------------------------------
0 | feature_encoder | AllCellFeatureEncoder | 1.2 K 
1 | backbone        | SCNWrapper            | 1.6 K 
2 | readout         | PropagateSignalDown   | 102   
3 | val_acc_best    | MeanMetric            | 0     
----------------------------------------------------------
2.9 K     Trainable params
0         Non-trainable params
2.9 K     Total params
0.012     Total estimated model params size (MB)


`Trainer.fit` stopped: `max_epochs=5` reached.


In [20]:
print('      Training metrics\n', '-'*26)
for key in train_metrics:
    print('{:<21s} {:>5.4f}'.format(key+':', train_metrics[key].item()))

      Training metrics
 --------------------------
train/accuracy:       0.2100
train/precision:      0.2096
train/recall:         0.2094
val/loss:             2.2357
val/accuracy:         0.1933
val/precision:        0.2936
val/recall:           0.1822
train/loss:           2.1610


## 6. Testing the model <a class="anchor" id="sec6"></a>

Finally, we can test the model and obtain the results.

In [21]:
%%capture
trainer.test(model, datamodule)
test_metrics = trainer.callback_metrics

In [22]:
print('      Testing metrics\n', '-'*25)
for key in test_metrics:
    print('{:<20s} {:>5.4f}'.format(key+':', test_metrics[key].item()))

      Testing metrics
 -------------------------
test/loss:           2.1464
test/accuracy:       0.2200
test/precision:      0.1581
test/recall:         0.2336
